In [1]:
import resfuncRead as rfr
import numpy as np
from Yield_calc import * 
import pandas as pd 

from tabulate import tabulate

bpar_er = rfr.getBandFunc('data/erband_R133') #reads in band data/fit for er data
bpar_nr = rfr.getBandFunc('data/nrband_R133')

ynr_mu = rfr.makeBFunc(bpar_nr[1]['mu'])
ynr_muv = np.vectorize(ynr_mu)
ynr_sig = rfr.makeBFunc(bpar_nr[1]['sig'],True)
ynr_sigv = np.vectorize(ynr_sig)

yer_mu = rfr.makeBFunc(bpar_er[1]['mu']) # sets average fit from 1st (2nd) col in data table. 
yer_muv = np.vectorize(yer_mu) #puts mean data for er in 1D array 
yer_sig = rfr.makeBFunc(bpar_er[1]['sig'],True) #sets uncertainty 
yer_sigv = np.vectorize(yer_sig)





In [2]:
s = 1
N = 10000
ER, Yield = Yield_NR(N)
#Erer, Yield_er = Yield_Er(N)

#cut unwanted data 
data = np.vstack((ER,Yield)).T
x = data[data[:,1]>=0, :]
c = x[x[:,0]>=10, :]
g = c[c[:,0]<=150, :]

Yield_nr = g[:,1]
Enr = g[:,0]


#define bands for nuclear recoil 
upper = ynr_mu(Enr)+s*ynr_sigv(Enr)
lower = ynr_mu(Enr)-s*ynr_sigv(Enr)


Data = np.vstack((Enr,Yield_nr,upper,lower)).T
Data1 = Data[np.argsort(Data[:, 0])]  #sorting data based on recoil energy. 



In [3]:
#binning data. 
df = pd.DataFrame(Data1)

bins  = [10,13.4,18.1,24.5,33.1,44.8,60.6,80.2,110,150]

df1 = pd.cut(df[0],bins, labels = False)


y = np.array(df[0].loc[df1 == 0])

print(y)



[10.0003348  10.00609801 10.01987465 10.0223459  10.02500852 10.02504241
 10.02686621 10.03594216 10.0493016  10.06903915 10.07216985 10.07982195
 10.08358965 10.09495169 10.0991853  10.1008297  10.13208983 10.13454601
 10.14085236 10.141571   10.14504983 10.15351037 10.18246581 10.19113388
 10.20244205 10.20303582 10.20498347 10.21795327 10.22164143 10.23574047
 10.23762832 10.2387498  10.24323353 10.24749087 10.25221739 10.2531845
 10.26180742 10.26432958 10.26550897 10.2948089  10.29616786 10.30212687
 10.3021939  10.30276355 10.30638567 10.32240552 10.34132833 10.35016699
 10.36470449 10.3772962  10.38920597 10.39778524 10.4036672  10.40444388
 10.40511892 10.41971246 10.42298546 10.43242602 10.44124757 10.4484865
 10.45282523 10.45352837 10.45394437 10.46188102 10.47905702 10.48316426
 10.50377682 10.50587181 10.51292116 10.52526368 10.52712811 10.53402477
 10.53727349 10.53810875 10.54246609 10.54514579 10.55657261 10.56496678
 10.56954763 10.57065152 10.5771528  10.57758306 10.5

In [4]:
Percent = []

for q in np.arange(0,9):
    
    count = []
    num = []
    count1 = []
    num1 = []
    
    
    x = np.array(df[1].loc[df1 == q]) #yield
    y = np.array(df[2].loc[df1 == q]) #upper band
    z = np.array(df[3].loc[df1 == q]) #lower band 
    
    
    for i,j in zip(x,y):
        if i > j:
            n = 1
        else: 
            n = 0 

        count.append(n)
    #print(count)

    for i in count: 
        if i == 1:
            num.append(i) 


    for i,j in zip(x,z):
        if i < j:
            n = 1
        else: 
            n = 0 

        count1.append(n)
    #print(count)

    for i in count1: 
        if i == 1:
            num1.append(i)


    N= len(x)
    print(N)
    percent = 100*(N - np.abs(len(num)+len(num1)))/N

    Percent.append(percent)


764
1035
1229
1331
1411
1458
1106
871
494


In [5]:
bin_spacing = '10-13.4','13.4-18.1','18.1-24.5','24.5-33.1','33.1-44.8','44.8-60.6','60.6-80.2','80.2-110','110-150'

In [6]:
print("Bin Spacing (keV)", '\t', "Percent in",s,"sigma")     #table column headings
print("--------------", '\t', '\t' "-------------------")

for x,y in zip(bin_spacing,Percent):
    print(x, '\t','\t', '{0:1.2f}'.format(y))


Bin Spacing (keV) 	 Percent in 1 sigma
-------------- 	 	-------------------
10-13.4 	 	 73.04
13.4-18.1 	 	 80.29
18.1-24.5 	 	 82.10
24.5-33.1 	 	 83.17
33.1-44.8 	 	 85.40
44.8-60.6 	 	 85.05
60.6-80.2 	 	 85.71
80.2-110 	 	 79.33
110-150 	 	 72.87
